** Header Code **

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels import discrete

import re
import pandas as pd
import math 
import csv
import time
import dateutil
from datetime import datetime
import seaborn as sns

from IPython.core.display import HTML
HTML("<style>.container {width:50% !important; }</style>");

C:\Users\nolefp\Anaconda\lib\site-packages\matplotlib\__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
# pandas options plus some more
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
pd.options.display.float_format = '{:,.2f}'.format
sns.set_style("whitegrid")
sns.set_context("poster")

In [3]:
# Matplotlib Formatting
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib import ticker


millnames = ['',' Thousand',' Million',' Billion',' Trillion']
def millify(n, pos):
    n = float(n)
    millidx = max(0,min(len(millnames)-1,
                        int(math.floor(0 if n == 0 else math.log10(abs(n))/3))))
    thingtoreturn = n / 10**(3 * millidx)
    if thingtoreturn % 1 == 0:
        return '{:.0f}{}'.format(thingtoreturn, millnames[millidx])
    elif thingtoreturn % 0.1 == 0:
        return '{:.1f}{}'.format(thingtoreturn, millnames[millidx])
    else:
        return '{:.2f}{}'.format(thingtoreturn, millnames[millidx])

In [4]:
municipal_codes_df = pd.read_excel("./exports/Municipal Code Matching.xlsx")

** General Pre-Processing Across NSO Data **

# Load Basic Datasets

## Population & Area Data

** Load and set-up **

In [5]:
# load the dataset
basic_populationdf = pd.read_excel("./Demographics/basic_population_area.xlsx")

In [6]:
# get rid of rows with null district (region totals, etc)
basic_populationdf.drop(basic_populationdf[basic_populationdf.mb.isnull()].index,inplace=1)

In [7]:
# explore dtpyptes and column names
print basic_populationdf.dtypes
print basic_populationdf.columns

mb                              float64
region_oblast                    object
povrsina_km2                      int64
broj_naselja                    float64
stanovnistvo_ukupno             float64
stanovnistvo_km2                float64
katarske_opstine                  int64
registrovane_mesne_zajednice    float64
mesne_kancelarije               float64
dtype: object
Index([u'mb', u'region_oblast', u'povrsina_km2', u'broj_naselja', u'stanovnistvo_ukupno', u'stanovnistvo_km2', u'katarske_opstine', u'registrovane_mesne_zajednice', u'mesne_kancelarije'], dtype='object')


In [8]:
# change variable types to int and float
basic_populationdf.mb = basic_populationdf.mb.astype(np.int32)
for var in [u'broj_naselja', u'stanovnistvo_ukupno', u'stanovnistvo_km2', u'registrovane_mesne_zajednice', u'mesne_kancelarije']:
    basic_populationdf[var] = basic_populationdf[var].astype(float, raise_on_error=False)


*Note from above, there are only 173 districts with complete population data. That is odd? It could be because of the "-" values that had to be removed. It also could be due to Kosovo. Let's see which ones they are.

It seems mostly to be due to Kosovo, aka not an issue since we aren't using it a lot.

** Translate to and properly headline ** 

In [9]:
basic_populationdf.columns = [u'mun_id', u'mun_cyr', u'area_km2', u'num_places', u'pop_total', 
                              u'pop_per_km2', u'katarske_opstine', u'registrovane_mesne_zajednice', u'mesne_kancelarije']

** Merging Population to demdf **

First, we perform the merge. 

In [10]:
demdf = municipal_codes_df.merge(basic_populationdf[[u'mun_id', u'mun_cyr', u'area_km2', u'pop_total']], how="left",on="mun_id")

** Examine types, missing vals, etc. **

In [11]:
demdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174 entries, 0 to 173
Data columns (total 6 columns):
mun_id       174 non-null int64
mun          174 non-null object
mun_type     174 non-null object
mun_cyr      173 non-null object
area_km2     173 non-null float64
pop_total    173 non-null float64
dtypes: float64(2), int64(1), object(3)
memory usage: 9.5+ KB


Comments:
- We see that population is missing for 5 entries. We exame that below.

- it turns out that uzice is the problem. Apparently the statistics office lists "Uzice City" and "Uzice" separately, which no one else does, and hence their is missing data on the subfield "Uzice". I replace the ID of Uzice City with that of Uzice and resolve the problem in this. The missing values should not occur the next time the dataset is loaded. 
- yup, worked out!
- beograd vracar is a duplicate and it is fine that it does not work
- nis-palilula idk, but it is a city district, so less important

## Employment

** Load and Set up **

In [12]:
employmentdf = pd.read_excel("./Demographics/age_sex_nationality_employment.xlsx","employment")

In [13]:
# get rid of rows with null district (region totals, etc)
employmentdf.drop(employmentdf[employmentdf['mun_id'].isnull()].index,inplace=1)

In [14]:
# change variable types to int and float
employmentdf.mun_id = employmentdf.mun_id.astype(np.int32)
for var in list(set(employmentdf.columns) - set(["mun_id", "mun"])):
    employmentdf[var] = employmentdf[var].astype(float)


In [15]:
employmentdf["labor_pariticpation_rate"] = employmentdf.active_total / employmentdf.total 
employmentdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 173 entries, 2 to 203
Data columns (total 16 columns):
mun_id                      173 non-null int32
mun                         173 non-null object
total                       173 non-null float64
active_total                173 non-null float64
active_employed             173 non-null float64
active_unemployed           173 non-null float64
active_unemp_former         173 non-null float64
active_unemp_firstjob       173 non-null float64
inactive_total              173 non-null float64
Unnamed: 9                  173 non-null float64
Unnamed: 10                 173 non-null float64
Unnamed: 11                 173 non-null float64
inactive_students           173 non-null float64
inactive_houshold           173 non-null float64
Unnamed: 14                 173 non-null float64
labor_pariticpation_rate    173 non-null float64
dtypes: float64(14), int32(1), object(1)
memory usage: 22.3+ KB


** merge to demdf**

In [16]:
cols = []
for x in employmentdf.columns:
    if not re.search("Unnam", x):
        cols.append(x)

demdf = demdf.merge(employmentdf[cols], how="outer",on="mun_id", suffixes=["","_empl"])

In [17]:
demdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174 entries, 0 to 173
Data columns (total 17 columns):
mun_id                      174 non-null int64
mun                         174 non-null object
mun_type                    174 non-null object
mun_cyr                     173 non-null object
area_km2                    173 non-null float64
pop_total                   173 non-null float64
mun_empl                    173 non-null object
total                       173 non-null float64
active_total                173 non-null float64
active_employed             173 non-null float64
active_unemployed           173 non-null float64
active_unemp_former         173 non-null float64
active_unemp_firstjob       173 non-null float64
inactive_total              173 non-null float64
inactive_students           173 non-null float64
inactive_houshold           173 non-null float64
labor_pariticpation_rate    173 non-null float64
dtypes: float64(12), int64(1), object(4)
memory usage: 24.5+ KB


## Nationalities

In [18]:
natdf = pd.read_excel("./Demographics/age_sex_nationality_employment.xlsx","nationality")

In [19]:
natdf.head()

,mun_id,mun,total,serbs,albanians,bosnians,bulgarians,bunjevci,vlasi,goranci,yugoslavs,hungarians,macedonians,muslims,germans,roma,romanians,russians,rusini,slovaks,slovenians,ukranians,croats,montenegrins,rest,undeclared,regional_assoc,unkown
0,nan,РЕПУБЛИКА СРБИЈА,"7,186,862.00","5,988,150.00","5,809.00","145,278.00","18,543.00","16,706.00","35,330.00","7,767.00","23,303.00","253,899.00","22,755.00","22,301.00","4,064.00","147,604.00","29,332.00","3,247.00","14,246.00","52,750.00","4,033.00","4,903.00","57,900.00","38,527.00","17,558.00","160,346.00","30,771.00","81,740.00"
1,nan,СРБИЈА – СЕВЕР,"3,591,249.00","2,795,083.00","3,503.00","2,376.00","2,677.00","16,641.00",352.00,"6,507.00","20,237.00","252,946.00","17,362.00","7,356.00","3,770.00","69,716.00","26,692.00","2,474.00","14,173.00","52,425.00","3,354.00","4,620.00","54,785.00","32,043.00","13,793.00","119,989.00","29,856.00","38,519.00"
2,"79,014.00",Београдски регион,"1,659,440.00","1,505,448.00","1,252.00","1,596.00","1,188.00",172.00,182.00,"5,328.00","8,061.00","1,810.00","6,970.00","3,996.00",498.00,"27,325.00","1,282.00","1,301.00",245.00,"2,104.00","1,539.00",418.00,"7,752.00","9,902.00","7,083.00","38,971.00","1,289.00","23,728.00"
3,nan,Београдска област\nГрад Београд,"1,659,440.00","1,505,448.00","1,252.00","1,596.00","1,188.00",172.00,182.00,"5,328.00","8,061.00","1,810.00","6,970.00","3,996.00",498.00,"27,325.00","1,282.00","1,301.00",245.00,"2,104.00","1,539.00",418.00,"7,752.00","9,902.00","7,083.00","38,971.00","1,289.00","23,728.00"
4,"70,092.00",Барајево,"27,110.00","25,496.00",16.00,7.00,9.00,nan,nan,28.00,36.00,10.00,85.00,35.00,6.00,252.00,10.00,12.00,2.00,7.00,10.00,6.00,55.00,109.00,32.00,266.00,7.00,614.00


In [20]:
# get rid of rows with null district (region totals, etc)
natdf.drop(natdf[natdf['mun_id'].isnull()].index,inplace=1)

In [21]:
# change variable types to int and float
natdf.mun_id = natdf.mun_id.astype(np.int32)

# explore dtpyptes and column names
natdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 171 entries, 2 to 203
Data columns (total 28 columns):
mun_id            171 non-null int32
mun               171 non-null object
total             171 non-null float64
serbs             171 non-null float64
albanians         153 non-null float64
bosnians          120 non-null float64
bulgarians        158 non-null float64
bunjevci          93 non-null float64
vlasi             95 non-null float64
goranci           99 non-null float64
yugoslavs         167 non-null float64
hungarians        158 non-null float64
macedonians       170 non-null float64
muslims           160 non-null float64
germans           132 non-null float64
roma              169 non-null float64
romanians         146 non-null float64
russians          159 non-null float64
rusini            106 non-null float64
slovaks           145 non-null float64
slovenians        148 non-null float64
ukranians         138 non-null float64
croats            170 non-null float64
mont

** export nationalities**

In [22]:
natdf_transform = natdf.drop(["mun","total"], axis=1).set_index("mun_id").stack()
natdf_transform.name = 'population'
natdf_transform.to_csv("./exports/nationalities_detailed.csv")

In [23]:
majority_nationalities_df = natdf_transform.reset_index().sort_values("population", ascending=0).groupby("mun_id").first()
majority_nationalities_df.columns = ["majority_ethnicity","spec_population"]
majority_nationalities_df.to_csv("./exports/majority_nationalities.csv")

In [24]:
natdf_transform.reset_index().groupby("level_1").population.sum().sort_values(ascending=0)[:10]

level_1
serbs            8,134,183.00
hungarians         269,110.00
undeclared         220,235.00
roma               189,236.00
bosnians           147,080.00
unkown             112,170.00
croats              71,519.00
slovaks             61,505.00
montenegrins        52,721.00
regional_assoc      41,916.00
Name: population, dtype: float64

** merge to demdf**

In [25]:
demdf = demdf.merge(natdf, how="outer",on="mun_id", suffixes=["","_nat"]).set_index("mun_id")
demdf["majority_nationality"] = majority_nationalities_df.majority_ethnicity

In [26]:
demdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174 entries, 70017 to 89010
Data columns (total 44 columns):
mun                         174 non-null object
mun_type                    174 non-null object
mun_cyr                     173 non-null object
area_km2                    173 non-null float64
pop_total                   173 non-null float64
mun_empl                    173 non-null object
total                       173 non-null float64
active_total                173 non-null float64
active_employed             173 non-null float64
active_unemployed           173 non-null float64
active_unemp_former         173 non-null float64
active_unemp_firstjob       173 non-null float64
inactive_total              173 non-null float64
inactive_students           173 non-null float64
inactive_houshold           173 non-null float64
labor_pariticpation_rate    173 non-null float64
mun_nat                     171 non-null object
total_nat                   171 non-null float64
serbs       

## Wages

In [27]:
wagedf = pd.read_excel("./Demographics/wages_employment.xlsx","wages")

In [28]:
# get rid of rows with null district (region totals, etc)
wagedf.drop(wagedf[wagedf['mun_id'].isnull()].index,inplace=1)

In [29]:
# change variable types to int and float
wagedf.mun_id = wagedf.mun_id.astype(np.int32)

# explore dtpyptes and column names
wagedf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175 entries, 2 to 211
Data columns (total 7 columns):
mun_id       175 non-null int32
mun          175 non-null object
wage_2010    167 non-null float64
wage_2011    169 non-null float64
wage_2012    173 non-null float64
wage_2013    173 non-null float64
wage_2014    173 non-null float64
dtypes: float64(5), int32(1), object(1)
memory usage: 10.3+ KB


** create annualized dataset**

In [40]:
wagedf_years = wagedf.drop("mun", axis=1).set_index("mun_id").stack().reset_index()
wagedf_years.columns = ["mun_id","year", "average_wage"]
wagedf_years["year"] = wagedf_years["year"].str.replace("wage_","").astype('int64') 
wagedf_years["average_wage"] = wagedf_years["average_wage"] / 100
wagedf_years.mun_id = wagedf_years.mun_id.astype('int64')
wagedf_years.set_index(["mun_id","year"], inplace=1)
wagedf_years.to_csv("./exports/ave_wage_per_year.csv")

** merge to demdf**

In [31]:
demdf = demdf.reset_index().merge(wagedf, how="outer",on="mun_id", suffixes=["","_wage"])

In [32]:
demdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 176 entries, 0 to 175
Data columns (total 51 columns):
mun_id                      176 non-null float64
mun                         174 non-null object
mun_type                    174 non-null object
mun_cyr                     173 non-null object
area_km2                    173 non-null float64
pop_total                   173 non-null float64
mun_empl                    173 non-null object
total                       173 non-null float64
active_total                173 non-null float64
active_employed             173 non-null float64
active_unemployed           173 non-null float64
active_unemp_former         173 non-null float64
active_unemp_firstjob       173 non-null float64
inactive_total              173 non-null float64
inactive_students           173 non-null float64
inactive_houshold           173 non-null float64
labor_pariticpation_rate    173 non-null float64
mun_nat                     171 non-null object
total_nat         

** get average wage changes year to year **

In [33]:
tuples = zip(['wage_2011', 'wage_2012', 'wage_2013', 'wage_2014'], ['wage_2010', 'wage_2011', 'wage_2012', 'wage_2013'])

In [34]:
def rel_wage_change(df, cols = ""):
    x11 = (df[diff[0]] == 0 | np.isnan(df[diff[0]])) 
    x13 = (df[diff[1]] == 0 | np.isnan(df[diff[1]]))
    if x11 & x13:
        return 0    
    if  ~x11 & x13:
        return -1
    if x11 &  ~x13:
        return 2
    else:
        return (df[diff[0]] - df[diff[1]])  /   df[diff[1]]

In [35]:
wagedf.set_index("mun_id", inplace=1)
wage_changes_abs = pd.DataFrame()
wage_changes_rel = pd.DataFrame()
for diff in tuples:
    wage_changes_abs[diff[0] + " - " + diff[1]] = wagedf[diff[0]] - wagedf[diff[1]]

for diff in tuples:
    wage_changes_rel[diff[0] + " - " + diff[1]] = wagedf.apply(rel_wage_change, cols=diff, axis=1)

In [36]:
wage_changes = pd.concat([wage_changes_abs.stack(), wage_changes_rel.stack()],axis=1)
wage_changes.reset_index(inplace=1)
wage_changes.columns = ["mun_id", "years_change", "wage_change_abs", "wage_change_rel"]
wage_changes.to_csv("./exports/changes_in_wages.csv")

## Add regions

In [37]:
regionsdf = pd.read_csv("./exports/munic_regional.csv")

In [38]:
test = demdf.merge(regionsdf.drop(["mun", "old", "area_id", "area"], axis=1), how='outer', on="mun_id") 
test

,mun_id,mun,mun_type,mun_cyr,area_km2,pop_total,mun_empl,total,active_total,active_employed,active_unemployed,active_unemp_former,active_unemp_firstjob,inactive_total,inactive_students,inactive_houshold,labor_pariticpation_rate,mun_nat,total_nat,serbs,albanians,bosnians,bulgarians,bunjevci,vlasi,goranci,yugoslavs,hungarians,macedonians,muslims,germans,roma,romanians,russians,rusini,slovaks,slovenians,ukranians,croats,montenegrins,rest,undeclared,regional_assoc,unkown,majority_nationality,mun_wage,wage_2010,wage_2011,wage_2012,wage_2013,wage_2014,region_id,region
0,"70,017.00",Aleksandrovac,normal,Александровац,387.00,"25,580.00",Александровац,"26,522.00","10,460.00","8,884.00","1,576.00",831.00,745.00,"16,062.00","2,276.00","2,786.00",0.39,Александровац,"26,522.00","25,682.00",5.00,1.00,7.00,nan,nan,nan,6.00,nan,10.00,1.00,nan,84.00,2.00,3.00,nan,nan,1.00,nan,8.00,23.00,9.00,71.00,nan,609.00,serbs,Александровац,"22,723.00","27,557.00","31,134.00","32,973.00","31,317.00",RS21,Region Sumadije i Zapadne Srbije
1,"70,025.00",Aleksinac,normal,Алексинац,707.00,"49,928.00",Алексинац,"51,863.00","19,232.00","12,909.00","6,323.00","3,512.00","2,811.00","32,631.00","3,373.00","5,744.00",0.37,Алексинац,"51,863.00","47,563.00",18.00,8.00,45.00,nan,21.00,13.00,49.00,16.00,98.00,37.00,6.00,"1,937.00",20.00,15.00,2.00,3.00,30.00,nan,50.00,68.00,35.00,474.00,9.00,"1,346.00",serbs,Алексинац,"27,703.00","30,710.00","32,531.00","36,032.00","36,954.00",RS22,Region Juzne i Istocne Srbije
2,"70,033.00",Arandjelovac,normal,Аранђеловац,376.00,"45,125.00",Аранђеловац,"46,225.00","18,148.00","13,721.00","4,427.00","2,755.00","1,672.00","28,077.00","3,475.00","4,182.00",0.39,Аранђеловац,"46,225.00","44,581.00",3.00,1.00,15.00,1.00,2.00,1.00,49.00,17.00,60.00,18.00,6.00,382.00,25.00,14.00,3.00,3.00,8.00,4.00,38.00,186.00,36.00,286.00,32.00,454.00,serbs,Аранђеловац,"29,348.00","32,092.00","35,968.00","37,517.00","40,324.00",RS21,Region Sumadije i Zapadne Srbije
3,"70,041.00",Arilje,normal,Ариље,349.00,"18,495.00",Ариље,"18,792.00","9,317.00","8,318.00",999.00,668.00,331.00,"9,475.00","1,575.00",849.00,0.50,Ариље,"18,792.00","18,407.00",2.00,4.00,8.00,nan,nan,nan,4.00,2.00,9.00,10.00,nan,120.00,nan,3.00,1.00,1.00,nan,5.00,8.00,22.00,9.00,132.00,nan,45.00,serbs,Ариље,"21,886.00","25,259.00","24,021.00","28,037.00","28,053.00",RS21,Region Sumadije i Zapadne Srbije
4,"70,050.00",Babusnica,normal,Бабушница,529.00,"11,478.00",Бабушница,"12,307.00","4,795.00","3,180.00","1,615.00",860.00,755.00,"7,512.00",659.00,"1,004.00",0.39,Бабушница,"12,307.00","10,933.00",1.00,1.00,632.00,nan,nan,nan,9.00,nan,5.00,1.00,3.00,244.00,3.00,nan,nan,nan,nan,nan,2.00,1.00,5.00,336.00,2.00,129.00,serbs,Бабушница,"27,603.00","26,849.00","28,105.00","28,843.00","32,205.00",RS22,Region Juzne i Istocne Srbije
5,"70,068.00",Bajina Basta,normal,Бајина Башта,673.00,"25,205.00",Бајина Башта,"26,022.00","11,834.00","10,148.00","1,686.00","1,119.00",567.00,"14,188.00","2,042.00","2,603.00",0.45,Бајина Башта,"26,022.00","25,638.00",8.00,1.00,1.00,nan,nan,1.00,17.00,3.00,13.00,9.00,1.00,1.00,1.00,10.00,nan,4.00,2.00,1.00,13.00,43.00,8.00,121.00,3.00,123.00,serbs,Бајина Башта,"30,460.00","31,839.00","35,512.00","36,754.00","36,356.00",RS21,Region Sumadije i Zapadne Srbije
6,"70,076.00",Batocina,normal,Баточина,136.00,"11,427.00",Баточина,"11,760.00","4,235.00","3,054.00","1,181.00",744.00,437.00,"7,525.00",803.00,"1,538.00",0.36,Баточина,"11,760.00","11,514.00",7.00,nan,1.00,nan,nan,nan,9.00,1.00,27.00,2.00,2.00,55.00,2.00,3.00,nan,3.00,nan,nan,6.00,18.00,9.00,74.00,3.00,24.00,serbs,Баточина,"24,930.00","27,127.00","28,759.00","28,732.00","29,604.00",RS21,Region Sumadije i Zapadne Srbije
7,"70,084.00",Bela Palanka,normal,Бела Паланка,517.00,"11,559.00",Бела Паланка,"12,126.00","4,296.00","2,639.00","1,657.00",994.00,663.00,"7,830.00",764.00,"1,161.00",0.35,Бела Паланка,"12,126.00","10,395.00",nan,nan,8.00,nan,nan,4.00,5.00,4.00,8.00,10.00,2.00,"1,418.00",nan,5.00,nan,1.00,nan,nan,4.00,5.00,8.0

# Calculate More Variables

Below we add:
- employment rate
- employed per total population
- portion serbian 

In [39]:
# employment rate
demdf["employment_rate"] =  demdf.active_employed / demdf.active_total

# employed per total pop
demdf["employment_per_pop"] = demdf.active_employed / demdf.pop_total

# portion serbia
demdf["share_serbian"] = demdf.serbs / demdf.total_nat

demdf["pop_density"] = demdf.pop_total / demdf.area_km2

** Regions, Proper Names and Type **

# Export

In [49]:
demdf.drop(["mun_cyr", "mun_empl","mun_nat", "mun_wage"], axis=1, inplace=1, errors='ignore')
demdf.mun_id = demdf.mun_id.astype(int)
demdf.to_csv("./exports/demdf.csv")

In [50]:
demdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 176 entries, 0 to 175
Data columns (total 51 columns):
mun_id                      176 non-null int32
mun                         174 non-null object
mun_type                    174 non-null object
area_km2                    173 non-null float64
pop_total                   173 non-null float64
total                       173 non-null float64
active_total                173 non-null float64
active_employed             173 non-null float64
active_unemployed           173 non-null float64
active_unemp_former         173 non-null float64
active_unemp_firstjob       173 non-null float64
inactive_total              173 non-null float64
inactive_students           173 non-null float64
inactive_houshold           173 non-null float64
labor_pariticpation_rate    173 non-null float64
total_nat                   171 non-null float64
serbs                       171 non-null float64
albanians                   153 non-null float64
bosnians         